In [1]:
# !pip install newspaper3k
# !pip install --upgrade beautifulsoup4
# !pip install beautifulsoup4
# %pip install joblib

In [2]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import HTML
from newspaper import Article
from datetime import datetime, date, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings("ignore")
from selenium.webdriver.common.by import By
import time
from joblib import Parallel, delayed
import tqdm 
from tqdm import tqdm
from functions import *

In [3]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

PATH=ChromeDriverManager().install()

In [4]:
def get_redactores_SP(url):
    try:
        article=Article(url)
        article.download()
        article.parse()
        return article.authors
    except:
        return 'Desconocido'

In [5]:
dia = str(date.today()).split('-')[2]
mes = str(date.today()).split('-')[1]
año = str(date.today()).split('-')[0][2:]

## Periódicos Digitales a scrappear:
### - Marca
### - AS
### - MundoDeportivo
### - Sport
### - EstadioDeportivo
### - SuperDeporte

# Escreapeamos MARCA

In [6]:
def add_fechas(link):
    try: 
        article=Article(link)
        article.download()
        article.parse()
        return article.publish_date.date()
    except:
        return 'Desconocido'

In [7]:
marca = 'https://www.marca.com/'

html = req.get(marca).text
soup = BeautifulSoup(html, "html.parser")

bodies = soup.find_all('div', class_='ue-c-cover-content__body')
titulares_raw = soup.find_all('h2', class_='ue-c-cover-content__headline')
titulares = [e.text for e in titulares_raw]

redactores_raw = [e.find_all('span', class_='ue-c-cover-content__byline-name') for e in bodies]
redactores = [e[0].text.split('\n')[1].strip().title() if len(e)>0 else 'Desconocido' for e in redactores_raw]

links = [e.find('a', class_='ue-c-cover-content__link').attrs['href'] for e in bodies]

comments_raw = [e.find('a', class_='ue-c-cover-content__comments') for e in bodies]
comments = [e.text.split('comentarios')[0] if e is not None else 'Sin Comentarios' for e in comments_raw]

print('Creando DataFrame...')
df_MARCA = pd.DataFrame()
df_MARCA['web'] = ['Marca' for i in range(len(titulares))]
df_MARCA['titular'] = titulares
df_MARCA['redactor'] = redactores
df_MARCA['comentarios'] = comments
df_MARCA['link'] = links
df_MARCA['noticia'] = ['Video' if 'videos.marca' in e else 'Noticia' for e in df_MARCA['link']] # Filtramos noticias válidas
try:
    df_MARCA['seccion'] = [e.split('.com/')[1].split('/')[0] for e in df_MARCA['link']] # Añadimos columna de sección
except:
    df_MARCA['seccion'] = 'Desconocido'
df_MARCA['seccion'] = ['video' if e == 'v' else 'Desconocido' if e == '' else e for e in df_MARCA['seccion']] # corregimos detalles 

# Añadimos las fechas de publicación    
fechas = Parallel(n_jobs=6, verbose=True)(delayed(add_fechas)(link) for link in tqdm(links))

df_MARCA['fecha_publicacion'] = fechas

df_MARCA['fecha_actual'] = [date.today() for e in range(len(titulares))]

# Añadimos margen de días entre actualidad y fecha de la noticia
desactualizacion = []
for i,e in enumerate(df_MARCA['fecha_publicacion']):
    if e != 'Desconocido':
        fecha1 = e
        fecha2 = df_MARCA['fecha_actual'][i]
        dias = (fecha2 - fecha1) / timedelta(days=1)
        desactualizacion.append(int(dias))
    else:
        desactualizacion.append(e)
        
df_MARCA['desactualizacion'] = desactualizacion

# df_MARCA = HTML(df.to_html(render_links=True)) # Hacemos los links clicables

df_MARCA.to_excel(f'./MARCA_{dia}_{mes}_{año}.xlsx')

df_MARCA.head()

Creando DataFrame...


100%|██████████| 65/65 [00:05<00:00, 11.00it/s]
[Parallel(n_jobs=6)]: Done  65 out of  65 | elapsed:    6.5s finished


,web,titular,redactor,comentarios,link,noticia,seccion,fecha_publicacion,fecha_actual,desactualizacion
0,Marca,Revolución en Inglaterra e Italia: ¿qué está p...,Jon Prada E Irati Prat,103,https://www.marca.com/futbol/futbol-internacio...,Noticia,futbol,2023-01-24,2023-01-24,0
1,Marca,Alves cayó en la trampa de la policía: así fue...,Desconocido,Sin Comentarios,https://us.marca.com/actualidad/2023/01/24/63c...,Noticia,actualidad,2023-01-24,2023-01-24,0
2,Marca,Danjuma lo tenía hecho con el Everton... ¡pero...,Aitana Sánchez,84,https://www.marca.com/futbol/mercado-fichajes/...,Noticia,futbol,2023-01-24,2023-01-24,0
3,Marca,La pieza clave para la llegada de Bellingham a...,Juan Ignacio García-Ochoa,197,https://www.marca.com/futbol/real-madrid/2023/...,Noticia,futbol,2023-01-24,2023-01-24,0
4,Marca,Reine-Adélaïde pasa el reconocimiento médico.....,A. Fdez.,30,https://www.marca.com/futbol/sevilla/2023/01/2...,Noticia,futbol,2023-01-24,2023-01-24,0


#### Probamos todo con Librería NewsPapper

In [8]:
# marca = 'https://www.marca.com/'

# html = req.get(marca).text
# soup = BeautifulSoup(html, "html.parser")

# links = [e.find('a', class_='ue-c-cover-content__link').attrs['href'] for e in bodies]

# df = pd.DataFrame()
# redactores = []
# titulos = []

# for e in links:
#     article=Article(e)
#     article.download()
#     article.parse()
    
#     redactores.append(article.authors)
#     titulos.append(article.title)
    
    
# comments_raw = [e.find('a', class_='ue-c-cover-content__comments') for e in bodies]
# comments = [e.text.split('comentarios')[0] if e is not None else 'Sin Comentarios' for e in comments_raw]
    
# df['web'] = ['Marca' for i in range(len(titulos))]
# df['titular'] = titulos
# df['redactor'] = redactores
# df['comentarios'] = comments
# df['link'] = links
# df['noticia'] = ['Video' if 'videos.marca' in e else 'Noticia' for e in df['link']] # Filtramos noticias válidas
# df['seccion'] = [e.split('.com/')[1].split('/')[0] for e in df['link']] # Añadimos columna de sección
# df['seccion'] = ['video' if e == 'v' else 'Desconocido' if e == '' else e for e in df['seccion']] # corregimos detalles

# # Añadimos la fecha
# fechas = []
# for e in links:
#     article=Article(e)
#     article.download()
#     article.parse()
#     fechas.append(article.publish_date)

# df['fecha'] = [e if e is None else datetime.strptime(str(e).split()[0], '%Y-%m-%d').date() for e in fechas]

# df

# Interesantes opciones, pero casi prefiero el ajuste manual porque los titulares con los de portada, los redactores los que
# firman, debo usar BS4 para los comentarios... nos quedamos con BS4

# Escreapeamos AS

In [9]:
def get_comments_AS(url):
    
    try:
        driver = webdriver.Chrome(PATH, options=opciones) #Para que abra la pestaña de Chrome
        driver.get(url) 

        time.sleep(1)

        driver.find_elements(By.XPATH, '//*[@id="didomi-notice-agree-button"]')[0].click() # Aceptamos cookies

        return int(driver.find_elements(By.XPATH, '//*[@id="top-num-comments"]')[0].text) # Nº de comentarios
    
    except:
        
        return 0
    driver.quit()

In [ ]:
AS = 'https://as.com/'

html = req.get(AS).text
soup = BeautifulSoup(html, "html.parser")

bodies = soup.find_all('article')

titulares_raw = soup.find_all('h2', class_='s__tl')
titulares = [e.text for e in titulares_raw]
print('Titulares cazados')

links = [e.find('a').attrs['href'] for e in bodies]
print('Links cazados')

# redactores_raw = [e.find_all('span', class_='s__au') for e in bodies]
# redactores = [e[0].text.title() for e in redactores_raw]

redactores = Parallel(n_jobs=6, verbose=True)(delayed(get_redactores_SP)(link) for link in tqdm(links))

print('Redactores cazados')


comments = Parallel(n_jobs=6, verbose=True)(delayed(get_comments_AS)(link) for link in tqdm(links))
print('Comentarios cazados')

print('Creando DataFrame...')
df_AS = pd.DataFrame()
df_AS['web'] = ['AS' for i in range(len(titulares))]
df_AS['titular'] = titulares
df_AS['redactor'] = redactores
df_AS['comentarios'] = comments
df_AS['link'] = links
df_AS['noticia'] = ['Video' if '/videos/' in e else 'Noticia' for e in df_AS['link']] # Filtramos noticias válidas

seccion = []
for e in links:
    if 'futbol' in e or 'mundial' in e or 'tikitaka' in e:
        seccion.append('Futbol')
    elif 'tenis' in e:
        seccion.append('Tenis')
    elif 'motor' in e:
        seccion.append('motor')
    elif '/videos/' in e:
        seccion.append(e.split('/videos/')[1].split('/')[0])
    elif '.com/' in e:
        seccion.append(e.split('.com/')[1].split('/')[0])
    else:
        seccion.append('Desconocido')
        
df_AS['seccion'] = seccion
print('¡Creado!')

# Añadimos las fechas de publicación    
print('Añadiendo fechas...')
fechas = Parallel(n_jobs=6, verbose=True)(delayed(add_fechas)(link) for link in tqdm(links))

df_AS['fecha_publicacion'] = fechas

df_AS['fecha_actual'] = [date.today() for e in range(len(titulares))]
print('Fechas añadidas!')

# Añadimos margen de días entre actualidad y fecha de la noticia
desactualizacion = []
for i,e in enumerate(df_AS['fecha_publicacion']):
    if e != 'Desconocido':
        fecha1 = e
        fecha2 = df_AS['fecha_actual'][i]
        dias = (fecha2 - fecha1) / timedelta(days=1)
        desactualizacion.append(int(dias))
    else:
        desactualizacion.append(e)
        
df_AS['desactualizacion'] = desactualizacion

print('¡ÉXITO!')

# df_AS = HTML(df.to_html(render_links=True)) # Hacemos los links clicables

df_AS.to_excel(f'./AS_{dia}_{mes}_{año}.xlsx')

df_AS.head()

Titulares cazados
Links cazados


100%|██████████| 139/139 [00:05<00:00, 27.09it/s]
[Parallel(n_jobs=6)]: Done 128 out of 139 | elapsed:    5.9s remaining:    0.4s
[Parallel(n_jobs=6)]: Done 139 out of 139 | elapsed:    7.0s finished


Redactores cazados


 82%|████████▏ | 114/139 [16:57<02:55,  7.03s/it]

# Escreapeamos MundoDeportivo

In [ ]:
def get_titles(url):
    try:
        article=Article(url)
        article.download()
        article.parse()
        return article.title
    except:
        return 'Desconocido'

In [ ]:
def get_comments_MD(url):
    
    try:
        driver = webdriver.Chrome(PATH, options=opciones) #Para que abra la pestaña de Chrome
        driver.get(url) 
        time.sleep(1)
        driver.find_elements(By.XPATH, '//*[@id="didomi-notice-agree-button"]')[0].click() # Aceptamos cookies
        return int(driver.find_elements(By.XPATH, '//*[@id="main-container"]/div[3]/article/div[1]/div[2]/div[2]/div[2]/div[2]/div[2]/a/span')[0].text) # Nº de comentarios
    
    except:
        return 0

In [ ]:
MD = 'https://www.mundodeportivo.com/'

html = req.get(MD).text
soup = BeautifulSoup(html, "html.parser")

bodies = soup.find_all('article')

verif = False

while verif == False:

    print('Cazando Titulares...')
    titulares = Parallel(n_jobs=6, verbose=True)(delayed(get_titles)(link) for link in tqdm(links))
    print('¡Cazados Titulares!')

    # Redactores NO EN PORTADA

    url_base = 'https://www.mundodeportivo.com/'

    links = []
    for e in bodies:
        try:
            links.append(e.find('a').attrs['href'])
        except:
            links.append('Desconocido')
    links = [e if 'http' in e else url_base+e for e in links]
    print(f'Cazados {len(links)} links')

    print('Cazando Redactores')
    redactores = Parallel(n_jobs=6, verbose=True)(delayed(get_redactores_MD)(link) for link in tqdm(links))
    
    verif = len(links) == len(titulares) == len(redactores)
        
print('Cazando Comments')
comments = Parallel(n_jobs=6, verbose=True)(delayed(get_comments_MD)(link) for link in tqdm(links))

print('Creando DataFrame...')
df_MD = pd.DataFrame()
df_MD['web'] = ['MD' for i in range(len(titulares))]
df_MD['titular'] = titulares
df_MD['redactor'] = redactores
df_MD['comentarios'] = comments
df_MD['link'] = links
df_MD['noticia'] = ['Video' if '/videos/' in e else 'Noticia' for e in df_MD['link']] # Filtramos noticias válidas
df_MD['seccion'] = [e.split('.com/')[1].split('/')[0] for e in df_MD['link']] # Añadimos columna de sección
print('¡Creado!')

# Añadimos las fechas de publicación    
print('Añadiendo fechas...')
fechas = Parallel(n_jobs=6, verbose=True)(delayed(add_fechas)(link) for link in tqdm(links))

df_MD['fecha_publicacion'] = fechas

df_MD['fecha_actual'] = [date.today() for e in range(len(titulares))]
print('Fechas añadidas!')

# Añadimos margen de días entre actualidad y fecha de la noticia
desactualizacion = []
for i,e in enumerate(df_MD['fecha_publicacion']):
    if e != 'Desconocido':
        fecha1 = e
        fecha2 = df_MD['fecha_actual'][i]
        dias = (fecha2 - fecha1) / timedelta(days=1)
        desactualizacion.append(int(dias))
    else:
        desactualizacion.append(e)
        
df_MD['desactualizacion'] = desactualizacion

print('¡ÉXITO!')

# df_MD = HTML(df.to_html(render_links=True)) # Hacemos los links clicables

df_MD.to_excel(f'./MundoDeportivo_{dia}_{mes}_{año}.xlsx')

df_MD.head()

# Escreapeamos SPORT

In [ ]:
def get_comments_SP(url):
    
    try:
        driver = webdriver.Chrome(PATH, options=opciones) #Para que abra la pestaña de Chrome
        driver.get(url) 
        time.sleep(1)
        driver.find_elements(By.XPATH, '//*[@id="didomi-notice-agree-button"]')[0].click() # Aceptamos cookies
        return int(driver.find_elements(By.XPATH, '//*[@id="infinite-scroll-details-item-1"]/div[2]/article/div[2]/div[1]/div[1]/div/div[2]/div[1]/div[2]/p/a[4]/span')[0].text) # Nº de comentarios
        driver.quit()
    
    except:
        return 0

    driver.quit()

In [ ]:
SP = 'https://www.sport.es/es/'

html = req.get(SP).text
soup = BeautifulSoup(html, "html.parser")

bodies = soup.find_all('article')

# Títulos con fallos por tildes. Cambiamos estrategia: Usamos librería Newspaper. Necesitamos links primero.

links = [e.find('a').attrs['href'] for e in bodies]
print('Cazados Links')

print('Cazando Titulares...')
titulares = Parallel(n_jobs=6, verbose=True)(delayed(get_titles)(link) for link in tqdm(links))
print('¡Cazados Titulares!')

redactores = Parallel(n_jobs=6, verbose=True)(delayed(get_redactores_SP)(link) for link in tqdm(links))
redactores = [e[0].title() if len(e)>0 else 'Desconocido' for e in redactores]

# print('Canzando Comentarios')
# comments = Parallel(n_jobs=6, verbose=True)(delayed(get_comments_SP)(link) for link in tqdm(links))

print('Creando DataFrame...')
df_SP = pd.DataFrame()
df_SP['web'] = ['SPORT' for i in range(len(titulares))]
df_SP['titular'] = titulares
df_SP['redactor'] = redactores
df_SP['comentarios'] = ['Desconocido' for i in range(len(titulares))]
df_SP['link'] = links
df_SP['noticia'] = ['Video' if '/videos/' in e else 'Noticia' for e in df_SP['link']] # Filtramos noticias válidas
seccion = []
for e in links:
    if 'futbol' in e or 'barca' in e or 'madrid' in e or 'seleccion' in e or 'mercado-fichajes' in e or 'mundial' in e:
        seccion.append('Futbol')
    elif 'psg' in e or 'espanyol' in e or 'girona' in e or 'fuera-de-juego' in e or 'segunda-division' in e or 'rayo-vallecano' in e:
        seccion.append('Futbol')
    elif 'euroliga' in e or 'nba' in e:
        seccion.append('baloncesto')
    elif '/noticias/' in e:
        seccion.append(e.split('/noticias/')[1].split('/')[0])
    elif '/videos/' in e:
        seccion.append(e.split('/videos/')[1].split('/')[0])
    elif '/es/' in e:
        seccion.append(e.split('/es/')[1].split('/')[0])
    elif '.es/' in e:
        seccion.append(e.split('.es/')[1].split('/')[0])
    elif '.com/' in e:
        seccion.append(e.split('.com/')[1].split('/')[0])
    else:
        seccion.append('Otros')
seccion = [e.replace('-',' ').title() for e in seccion]
df_SP['seccion'] = seccion
print('¡Creado!')

# Añadimos las fechas de publicación    
print('Añadiendo fechas...')
fechas = Parallel(n_jobs=6, verbose=True)(delayed(add_fechas)(link) for link in tqdm(links))

df_SP['fecha_publicacion'] = fechas

df_SP['fecha_actual'] = [date.today() for e in range(len(titulares))]
print('Fechas añadidas!')

# Añadimos margen de días entre actualidad y fecha de la noticia
desactualizacion = []
for i,e in enumerate(df_SP['fecha_publicacion']):
    if e != 'Desconocido':
        fecha1 = e
        fecha2 = df_SP['fecha_actual'][i]
        dias = (fecha2 - fecha1) / timedelta(days=1)
        desactualizacion.append(int(dias))
    else:
        desactualizacion.append(e)
        
df_SP['desactualizacion'] = desactualizacion

print('¡ÉXITO!')

# df_MD = HTML(df.to_html(render_links=True)) # Hacemos los links clicables

df_SP.to_excel(f'./SPORT_{dia}_{mes}_{año}.xlsx')

df_SP.head()

# Escreapeamos EstadioDeportivo

In [ ]:
def get_redactores_ED(url):
    try:
        driver = webdriver.Chrome(PATH, options=opciones) #Para que abra la pestaña de Chrome
        driver.get(url) 
        time.sleep(2)
        driver.find_elements(By.XPATH, '//*[@id="acceptAllMain"]')[0].click() # Aceptamos cookies
        redactor = driver.find_elements(By.XPATH, '/html/body/div[2]/span[2]/span[2]/span[1]/span[1]/span[2]/span[1]/a[1]')[0].text.title()
        driver.quit()
        return redactor
    except:
        try:
            driver = webdriver.Chrome(PATH, options=opciones) #Para que abra la pestaña de Chrome
            driver.get(url) 
            time.sleep(2)
            driver.find_elements(By.XPATH, '//*[@id="acceptAllMain"]')[0].click() # Aceptamos cookies
            redactor = driver.find_elements(By.XPATH, '/html/body/div[2]/span[2]/span[2]/span[1]/span[2]/span[2]/span[1]/a[1]')[0].text.title()
            driver.quit()
            return redactor
        except:
            try:
                driver = webdriver.Chrome(PATH, options=opciones) #Para que abra la pestaña de Chrome
                driver.get(url) 
                time.sleep(2)
                driver.find_elements(By.XPATH, '//*[@id="acceptAllMain"]')[0].click() # Aceptamos cookies
                redactor = driver.find_elements(By.XPATH, '/html/body/div[2]/span[2]/span[2]/span[1]/span[1]/span/a')[0].text.title()
                driver.quit()
                return redactor
            except:
                return 'Desconocido'
            
    driver.quit()

In [ ]:
ED = 'https://www.estadiodeportivo.com/'

html = req.get(ED).text
soup = BeautifulSoup(html, "html.parser")

bodies = soup.find_all('article')

links = [e.find('a').attrs['href'] for e in bodies]
print('Cazados Links')

print('Cazando Titulares...')
titulares = Parallel(n_jobs=6, verbose=True)(delayed(get_titles)(link) for link in tqdm(links))
print('¡Cazados Titulares!')

redactores = Parallel(n_jobs=6, verbose=True)(delayed(get_redactores_ED)(link) for link in tqdm(links))

print('Creando DataFrame...')
df_ED = pd.DataFrame()
df_ED['web'] = ['EstadioDeportivo' for i in range(len(titulares))]
df_ED['titular'] = titulares
df_ED['redactor'] = redactores
df_ED['comentarios'] = ['Desconocido' for i in range(len(titulares))]
df_ED['link'] = links
df_ED['noticia'] = ['Video' if '/videos/' in e else 'Noticia' for e in df_ED['link']] # Filtramos noticias válidas
df_ED['seccion'] = [e.split('.com/')[1].split('/')[0] for e in links]

print('¡Creado!')

# Añadimos las fechas de publicación    
print('Añadiendo fechas...')
fechas = Parallel(n_jobs=6, verbose=True)(delayed(add_fechas)(link) for link in tqdm(links))

df_ED['fecha_publicacion'] = fechas

df_ED['fecha_actual'] = [date.today() for e in range(len(titulares))]
print('Fechas añadidas!')

# Añadimos margen de días entre actualidad y fecha de la noticia
desactualizacion = []
for i,e in enumerate(df_ED['fecha_publicacion']):
    if e != 'Desconocido':
        fecha1 = e
        fecha2 = df_ED['fecha_actual'][i]
        dias = (fecha2 - fecha1) / timedelta(days=1)
        desactualizacion.append(int(dias))
    else:
        desactualizacion.append(e)
        
df_ED['desactualizacion'] = desactualizacion

print('¡ÉXITO!')

# df_MD = HTML(df.to_html(render_links=True)) # Hacemos los links clicables

df_ED.to_excel(f'./EstadioDeportivo_{dia}_{mes}_{año}.xlsx')

df_ED.head()

# Escreapeamos SuperDeporte

In [ ]:
def get_redactores_Super(url):
    try:
        article=Article(url)
        article.download()
        article.parse()
        return article.authors[0]
    except:
        return 'Desconocido'

In [ ]:
def get_comments_Super(url):
    try:
        driver = webdriver.Chrome(PATH, options=opciones) #Para que abra la pestaña de Chrome
        driver.get(url) 
        time.sleep(1)
        driver.find_elements(By.XPATH, '/html/body/div[1]/div/div/div/div/div/div[2]/button[2]')[0].click() # Aceptamos cookies
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        return int(driver.find_elements(By.XPATH, '/html/body/main/div[5]/div/div/div/article/div/div')[0].text.split()[0]) # Nº de comentarios
    except:
        return 0

In [ ]:
SUPER = 'https://www.superdeporte.es/'

html = req.get(SUPER).text
soup = BeautifulSoup(html, "html.parser")

bodies = soup.find_all('article')

links = []
for e in bodies:
    try:
        if 'https://' in e.find('a').attrs['href']:
            links.append(e.find('a').attrs['href'].strip())
        else:
            links.append('https://www.superdeporte.es/'+e.find('a').attrs['href'].strip())
    except:
        links.append('Sin Link')
print('Cazados Links')

print('Cazando Titulares...')
titulares = Parallel(n_jobs=6, verbose=True)(delayed(get_titles)(link) for link in tqdm(links))
print('¡Cazados Titulares!')

redactores = Parallel(n_jobs=6, verbose=True)(delayed(get_redactores_Super)(link) for link in tqdm(links))
print('Redactores cazados')

# comments = Parallel(n_jobs=6, verbose=True)(delayed(get_comments_Super)(link) for link in tqdm(links))

print('Creando DataFrame...')
df_Super = pd.DataFrame()
df_Super['web'] = ['SuperDeporte' for i in range(len(titulares))]
df_Super['titular'] = titulares
df_Super['redactor'] = redactores
df_Super['comentarios'] = ['Desconocido' for i in range(len(titulares))]
df_Super['link'] = links
df_Super['noticia'] = ['Video' if '/videos/' in e else 'Noticia' for e in df_Super['link']] # Filtramos noticias válidas
seccion = []
for e in links:
    try:
        seccion.append(e.split('.es//')[1].split('/')[0])
    except:
        seccion.append('Otros')
df_Super['seccion'] = seccion
print('¡Creado!')

# Añadimos las fechas de publicación    
print('Añadiendo fechas...')
fechas = Parallel(n_jobs=6, verbose=True)(delayed(add_fechas)(link) for link in tqdm(links))

df_Super['fecha_publicacion'] = fechas

df_Super['fecha_actual'] = [date.today() for e in range(len(titulares))]
print('Fechas añadidas!')

# Añadimos margen de días entre actualidad y fecha de la noticia
desactualizacion = []
for i,e in enumerate(df_Super['fecha_publicacion']):
    if e != 'Desconocido':
        fecha1 = e
        fecha2 = df_Super['fecha_actual'][i]
        dias = (fecha2 - fecha1) / timedelta(days=1)
        desactualizacion.append(int(dias))
    else:
        desactualizacion.append(e)
        
df_Super['desactualizacion'] = desactualizacion

print('¡ÉXITO!')

# df_MD = HTML(df.to_html(render_links=True)) # Hacemos los links clicables

df_Super.to_excel(f'./Super_{dia}_{mes}_{año}.xlsx')

df_Super.head()